This file runs model inference for a small very small, 250M parameter model.

In [ ]:
!pip install transformers
!pip install accelerate
!pip install google-colab
!pip install -q -U bitsandbytes
!pip install torch torch_xla
!pip install einops

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="crumb/d1536-250MT-full", trust_remote_code=True)

In [3]:
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, Accelerator
import torch
import torch_xla
import torch_xla.core.xla_model as xm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("crumb/d1536-250MT-full", trust_remote_code=True)
name = 'crumb/d1536-250MT-full'

In [ ]:
# get tokenizer
tokenizer = AutoTokenizer.from_pretrained(name)
# inputs
input_text = """

1. The food was great quality. Their tofu was very great. The size of each dish is pretty big so you'll leave with a full stomach. The waiting time was a few minutes, I was impressed by how fast the service was.

2. Just had the Pad Thai, hands down the worst Pad Thai I have ever had. I have had their Pad Thai in their other locations and it was very good. Not sure what happened here tonight, but it was a real bummer. Table Service was great, prompt and fast.

3. Just a short walking distance from the UT campus. The restaurant has indoor and outdoor seating. A bit pricy but the quality of the food really compensates for it.

4. Been searching Austin for a while now to find the best Thai food, and was recommended Madam Man's by a friend. Unfortunately I still have not found the best Thai yet. We ordered to-go and it was ready for pickup upon arrival. It looked like a lot of orders are pickup vs dine in (but this was a Sunday night). The restaurant looked like it had a nice patio but a bit of a sparse interior. For food we ordered Pad See Ew, pad Thai and Thai tea. Didn't love any of these, but I have heard their Thai chicken wings are very good so I may try again. The pad Thai was overly "sauce" flavored but yet somehow still dry. Tasted more like lime than peanut or anything I'm used to. The Thai tea was very watery (but it had been sitting for a while waiting on us). The pad see ew was the best dish we had and was ordered as a reviewer had recommended (chicken, extra sauce, spicy 3). First of all, if their spicy scale was supposed to be out of 10 (I asked when ordering), that was a pretty darn spicy 3. The noodles did not seem like they had extra sauce (they seemed to have very little sauce at all) - overall just not what I was hoping for. I love places on Campus like this and want to try some of their other things, but first time wasn't the charm.

"""

categorized_prompt = """

Please provide a detailed, objective summary for each of the following categories based on the provided restaurant reviews: Ambiance, Food, and Service. Ensure each summary focuses on different aspects as mentioned in the reviews, avoiding repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions in each category.

"""

general_prompt = """

Please provide a one-sentence, objective summary for the restaurant based on the provided reviews. Ensure the summary focuses on different aspects as mentioned in the reviews, avoiding repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions.

"""

ambiance_prompt = """

Please provide a one-sentence, objective summary for the "ambiance" aspect of the restaurant based on the provided reviews. Ensure the summary avoids repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions.

"""

food_prompt = """

Please provide a one-sentence, objective summary for the "food" aspect of the restaurant based on the provided reviews. Ensure the summary avoids repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions.

"""

service_prompt = """

Please provide a one-sentence, objective summary for the "service" aspect of the restaurant based on the provided reviews. Ensure the summary avoids repetition and first-person narratives. Please do not include numerical ratings, but rather focus on qualitative descriptions that clearly convey the reviewers' experiences and opinions.

"""


input_ids = tokenizer.encode(input_text + general_prompt, return_tensors='pt')
input_ids2 = tokenizer.encode(input_text + ambiance_prompt, return_tensors='pt')
input_ids3 = tokenizer.encode(input_text + food_prompt, return_tensors='pt')
input_ids4 = tokenizer.encode(input_text + service_prompt, return_tensors='pt')
# input_ids = accelerator.prepare(input_ids)

In [6]:
input_ids = input_ids.to("cuda")
input_ids.device

attention_mask = torch.ones(input_ids.shape).to(torch.float16)
attention_mask = attention_mask.to('cuda')
# model.device
# attention_mask.dtype

In [ ]:
# generate text

# from transformers import quantized_inference
# attention_mask = torch.ones(input_ids.shape).to(torch.float16)

# attention_mask = accelerator.prepare(attention_mask)

# input_ids.cuda()
# attention_mask.cuda()
# from transformers import quantized_inference

with torch.no_grad():
  model.cuda()
  output = model.generate(input_ids,
              attention_mask=attention_mask,
              max_length=1000,
              do_sample=True,
              top_k=10,
              num_return_sequences=1,
              eos_token_id=tokenizer.eos_token_id)



  output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)